In [ ]:
import pandas as pd
from detect_and_score import *
from scraper import *

import pandas as pd
import numpy as np
import nltk
import csv
from nltk.corpus import stopwords

In [ ]:
# load the word2vec model
word2vec = gensim.models.KeyedVectors.load_word2vec_format('models/GoogleNews-vectors-negative300.bin', binary=True)  

In [ ]:
# Load the argument quality model
score_model = keras.models.load_model("models/argument_quality.h5")
detect_model = keras.models.load_model("models/bilstm_cos.h5")

In [ ]:
# extract reddit topics with posts
reddit_data = makeDF(range(0, 300, 1))

In [ ]:
# report amount of proper topics found
print("Amount of topics:", len(reddit_data))

In [ ]:
from random import randrange
def test_on_reddit(detect_model, threshold, score_model, reddit_data, print_results = False):

    topic_accuracies = []
    all_topics = []
    for i in range(len(reddit_data)):
        print("---------------------------Processing topic: {0}-----------------------------------".format(i))
        # get the delta posts
        delta_posts = reddit_data[i][3]
        normal_posts = reddit_data[i][2]
        n_delta_posts = len(delta_posts)
        # get the normal posts, WITHOUT the delta posts
        normal_posts_filtered = []
        for normal_post in normal_posts:
            is_delta = False
            for delta_post in delta_posts:
                if(normal_post == delta_post):
                    is_delta = True

            if(is_delta == False):
                normal_posts_filtered.append(normal_post)

        n_normal_posts = len(normal_posts_filtered)     

        # get topic and store for later
        topic = reddit_data[i][0]
        # score normal posts
        normal_scores = []

        for post in normal_posts_filtered:
            score = detect_and_score(detect_model, threshold, score_model, post, topic, word2vec, print_results)
            normal_scores.append(score)
    
        print("Normal post scores:", normal_scores)
    
    
    
    
        """delta_posts = []
        rand_idx = randrange(0, len(normal_posts_filtered))
        delta_posts.append(normal_posts_filtered[rand_idx])"""
        

           
        
        
        # score delta posts
        delta_scores = []
        for post in delta_posts:
            score = detect_and_score(detect_model, threshold, score_model, post, topic, word2vec, print_results)
            delta_scores.append(score)
            
        print("Delta post scores:", delta_scores)

        # check for each delta post how many normal scores are lower than its own score
        total_accuracy = 0
        for delta_score in delta_scores:
            correct = 0
            for normal_score in normal_scores:
                if(delta_score > normal_score):
                    correct += 1

            accuracy = correct / n_normal_posts
            
                
            total_accuracy += accuracy
        total_accuracy /= n_delta_posts

        print("Topic", i, ":", total_accuracy)
        all_topics.append(topic)
        topic_accuracies.append(total_accuracy)
    return topic_accuracies, all_topics

In [ ]:
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]

In [ ]:
for threshold in thresholds:
    print("--------------------------------threshold = {0}---------------------------------------------".format(threshold))
    topic_accuracies, all_topics = test_on_reddit(detect_model, threshold, score_model, reddit_data)
    mean = np.mean(np.array(topic_accuracies))
    variance = np.var(np.array(topic_accuracies))
    print("Mean Accuracy: {0} \n Variance: {1}".format(mean, variance))

In [ ]:
topic_accuracies, all_topics = test_on_reddit(detect_model, 0.1, score_model, reddit_data, True)

In [ ]:
all_topics

In [ ]:
topic_accuracies

In [ ]:
len(topic_accuracies)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(range(len(topic_accuracies)), topic_accuracies)
plt.xlabel("Topic")
plt.ylabel("Accuracy")
plt.title("Mean topic accuracy, threshold = 0.1")
plt.show()